# A

In [91]:
import numpy as np
from PIL import Image
import os
from scipy.signal import convolve2d
from scipy.optimize import minimize
import itertools
import csv
import warnings


# --- Configuration ---
DATA_DIR =  './best_pictures'
KNOWN_FILTERS_FILE = os.path.join(DATA_DIR, 'algos.csv')
OUTPUT_CSV_FILE = 'reconstructed_algos.csv'

# можно увеличить для большей точности
MAX_PAIRS_FOR_OPTIMIZATION = 10

MAXITER = 20000

In [92]:
def load_known_filters(filepath):
    """Загружает известные фильтры из CSV файла."""
    try:
        filters_flat = np.loadtxt(filepath, delimiter=',')
        if filters_flat.shape != (2, 9):
            raise ValueError("CSV файл должен содержать ровно 2 строки по 9 чисел.")
        # Преобразуем в формат 3x3
        filters = [f.reshape(3, 3) for f in filters_flat]
        print(f"Загружены 2 известных фильтра из {filepath}")
        return filters
    except Exception as e:
        print(f"Ошибка при загрузке известных фильтров из {filepath}: {e}")
        print("Убедитесь, что файл существует и имеет правильный формат.")
        return None

In [93]:
def find_image_output_pairs(data_dir):
    """Находит пары файлов входного изображения (.png) и выходной матрицы (.txt)."""
    pairs = []
    files = os.listdir(data_dir)
    png_files = sorted([f for f in files if f.lower().endswith('.png')])
    for png_file in png_files:
        base_name = os.path.splitext(png_file)[0]
        txt_file = base_name + '.txt'
        if txt_file in files:
            pairs.append({
                'image': os.path.join(data_dir, png_file),
                'output': os.path.join(data_dir, txt_file)
            })
    print(f"Найдено {len(pairs)} пар вход/выход.")
    if not pairs:
        print(f"В папке {data_dir} не найдено пар .png/.txt файлов.")
    return pairs

In [94]:
def load_image_grayscale(filepath):
    """Загружает изображение и конвертирует в одноканальный numpy массив (float)."""
    try:
        img = Image.open(filepath).convert('L') # 'L' для grayscale
        return np.array(img, dtype=np.float64)
    except Exception as e:
        print(f"Ошибка при загрузке изображения {filepath}: {e}")
        return None

In [95]:
def load_output_matrix(filepath):
    """Загружает выходную матрицу из .txt файла."""
    try:
        return np.loadtxt(filepath, dtype=np.float64)
    except Exception as e:
        print(f"Ошибка при загрузке выходной матрицы {filepath}: {e}")
        return None

In [96]:
def calculate_mse(y_true, y_pred):
    """Рассчитывает среднеквадратичную ошибку."""
    return np.mean((y_true - y_pred) ** 2)

In [97]:
# --- Helper Functions ---
def apply_filter(image, kernel):
    """
    Применяет фильтр 3x3 к изображению методом скользящего окна.
    Использует 'valid' свертку (без дополнения краев).
    """
    # Убедимся, что ядро имеет правильную форму
    if kernel.shape != (3, 3):
        raise ValueError("Ядро фильтра должно быть размером 3x3")
    # scipy.signal.convolve2d с mode='valid' делает то, что нужно:
    # поэлементное произведение окна и фильтра -> сумма
    # Важно: переворачиваем ядро, т.к. convolve2d выполняет свертку,
    # а в описании задачи - корреляция (или свертка с перевернутым ядром).
    # Для симметричных ядер это не имеет значения, но лучше сделать правильно.
    return convolve2d(image, np.rot90(kernel, 2), mode='same')

def apply_sequence(image, filters):
    """Последовательно применяет список фильтров к изображению."""
    current_image = image.copy()
    for f in filters:
        current_image = apply_filter(current_image, f)
    return current_image



# --- Основная логика ---

def reconstruct_kainet(data_dir, known_filters_file, output_csv_file):
    """
    Главная функция для восстановления фильтров KaiNet.
    """
    print("--- Начало восстановления KaiNet ---")

    # 1. Загрузка известных фильтров
    known_filters = load_known_filters(known_filters_file)
    if known_filters is None:
        return
    f1, f2 = known_filters

    # 2. Поиск и загрузка данных (ограниченное количество пар)
    image_output_pairs = find_image_output_pairs(data_dir)
    if not image_output_pairs:
        return

    loaded_data = []
    print(f"Загрузка данных для оптимизации (макс. {MAX_PAIRS_FOR_OPTIMIZATION} пар)...")
    for i, pair in enumerate(image_output_pairs):
        if i >= MAX_PAIRS_FOR_OPTIMIZATION:
            break
        img = load_image_grayscale(pair['image'])
        output_matrix = load_output_matrix(pair['output'])

        if img is not None and output_matrix is not None:
            loaded_data.append({'image': img, 'output': output_matrix})
            print(f"  Загружена пара: {os.path.basename(pair['image'])} / {os.path.basename(pair['output'])}")

        else:
            print(f"  Пропуск пары из-за ошибки загрузки.")

    if not loaded_data:
        print("Не удалось загрузить ни одной валидной пары вход/выход для оптимизации. Прерывание.")
        return

    # 3. Перебор перестановок и оптимизация
    filters_to_permute = [f1, f2, 'unknown'] # Метка для неизвестного фильтра
    possible_permutations = list(itertools.permutations(filters_to_permute))
    print(f"\nНачинаем проверку {len(possible_permutations)} возможных порядков фильтров...")

    best_result = {
        'final_filters': None, # Сохраняем итоговый набор фильтров
        'unknown_filter': None,
        'mse': float('inf'),
        'order_description': None
    }

    # Используем метод оптимизации, который не требует градиентов (может быть медленнее, но проще)
    # 'Nelder-Mead' - хороший выбор для начала.
    optimization_method = 'Nelder-Mead'
    # Начальное предположение для неизвестного фильтра (9 значений) - нули
    initial_guess_unknown = np.zeros(9)

    for i, permutation in enumerate(possible_permutations):
        # Создаем читаемое описание порядка для вывода
        order_desc_list = []
        known_filter_indices = {id(f1): 'F1', id(f2): 'F2'} # Используем id для различения f1 и f2
        for item in permutation:
            if isinstance(item, str) and item == 'unknown':
                order_desc_list.append('F_unknown')
            elif id(item) in known_filter_indices:
                 order_desc_list.append(known_filter_indices[id(item)])
            else:
                 order_desc_list.append('?') # На всякий случай
        order_description_str = " -> ".join(order_desc_list)

        print(f"\n--- Проверка порядка #{i+1}: {order_description_str} ---")


        # Функция потерь для оптимизатора
        def loss_function(unknown_filter_flat):
            unknown_filter_matrix = unknown_filter_flat.reshape(3, 3)
            current_mse_sum = 0
            num_valid_pairs = 0

            # Собираем полный список фильтров для этой перестановки
            current_filter_sequence = []
            for item in permutation:
                # *** ИСПРАВЛЕНИЕ ЗДЕСЬ ***
                # Проверяем, является ли item строкой 'unknown'
                if isinstance(item, str) and item == 'unknown':
                    current_filter_sequence.append(unknown_filter_matrix)
                else:
                    # Иначе это должен быть один из известных фильтров (NumPy array)
                    current_filter_sequence.append(item)

            # Считаем MSE по всем загруженным парам
            for data_pair in loaded_data:
                input_img = data_pair['image']
                target_output = data_pair['output']

                # Применяем последовательность фильтров
                try:
                    calculated_output = apply_sequence(input_img, current_filter_sequence)

                    # Проверяем совпадение размеров выхода (хотя уже проверили при загрузке)
                    if calculated_output.shape != target_output.shape:
                        warnings.warn(f"Размер вычисленного выхода {calculated_output.shape} "
                                      f"не совпадает с целевым {target_output.shape} для порядка {order_description_str}. "
                                      f"Это не должно было произойти. Пропуск пары.")
                        continue

                    current_mse_sum += calculate_mse(target_output, calculated_output)
                    num_valid_pairs += 1
                except Exception as e:
                     # Добавим больше деталей в сообщение об ошибке
                     warnings.warn(f"Ошибка при применении фильтров для порядка {order_description_str} "
                                   f"на изображении {data_pair.get('image_path', 'N/A')}: {e}. Пропуск пары.")
                     continue

            if num_valid_pairs == 0:
                 warnings.warn(f"Не удалось вычислить MSE ни для одной пары для порядка {order_description_str}.")
                 return float('inf') # Возвращаем бесконечность, если ни одна пара не сработала

            average_mse = current_mse_sum / num_valid_pairs
            # print(f"    Текущий MSE: {average_mse:.6f}") # Отладочный вывод (может быть много)
            return average_mse

        # Запускаем оптимизацию для поиска лучшего unknown_filter для этого порядка
        print(f"  Запуск оптимизации ({optimization_method}) для поиска неизвестного фильтра...")
        opt_result = minimize(loss_function,
                              initial_guess_unknown,
                              method=optimization_method,
                              options={'maxiter': MAXITER, 'adaptive': True, 'fatol': 1e-6, 'xatol': 1e-6}) # Увеличим maxiter и добавим tolerance

        if opt_result.success or 'converged' in opt_result.message.lower(): # Считаем схождение тоже успехом
            found_unknown_filter = opt_result.x.reshape(3, 3)
            final_mse = opt_result.fun
            print(f"  Оптимизация завершена. Минимальный MSE для этого порядка: {final_mse:.8f}")
            if not opt_result.success:
                 print(f"  (Примечание: Оптимизация формально не 'успешна', но сошлась: {opt_result.message})")


            # Обновляем лучший результат, если текущий лучше
            if final_mse < best_result['mse']:
                print(f"  *** Найден новый лучший результат! ***")
                best_result['mse'] = final_mse
                best_result['unknown_filter'] = found_unknown_filter
                best_result['order_description'] = order_description_str # Сохраняем читаемое описание
                # Сохраняем реальные фильтры в найденном порядке
                final_order_filters = []
                for item in permutation:
                   if isinstance(item, str) and item == 'unknown':
                       final_order_filters.append(found_unknown_filter)
                   else:
                       final_order_filters.append(item)
                best_result['final_filters'] = final_order_filters

        else:
            print(f"  Оптимизация не сошлась для этого порядка. Причина: {opt_result.message}")

    # 4. Вывод и сохранение результата
    print("\n--- Результаты реконструкции ---")
    if best_result['final_filters'] is not None:
        print(f"Найден лучший порядок фильтров: {best_result['order_description']}")
        print(f"Минимальное среднее MSE: {best_result['mse']:.8f}")
        print("Найденный неизвестный фильтр (F_unknown):")
        # Используем printoptions для красивого вывода матрицы
        with np.printoptions(precision=6, suppress=True):
             print(best_result['unknown_filter'])

        # Сохраняем все три фильтра в правильном порядке
        print(f"\nСохранение восстановленных фильтров в файл: {output_csv_file}")
        try:
            with open(output_csv_file, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                for f in best_result['final_filters']:
                    # Сохраняем как строку из 9 чисел
                    writer.writerow(np.round(f.flatten(), 8)) # Сохраняем с некоторой точностью
            print("Файл успешно сохранен.")
            print("\nСодержимое файла (первые 3 строки):")
            with open(output_csv_file, 'r') as f:
                for i, line in enumerate(f):
                    if i >= 3: break
                    print(line.strip())


        except Exception as e:
            print(f"Ошибка при сохранении файла {output_csv_file}: {e}")

    else:
        print("Не удалось найти решение. Возможно, стоит попробовать:")
        print("- Увеличить MAX_PAIRS_FOR_OPTIMIZATION (если есть больше пар).")
        print("- Попробовать другие методы оптимизации ('BFGS', 'CG', 'L-BFGS-B').")
        print("- Попробовать другие начальные приближения для 'initial_guess_unknown'.")
        print("- Увеличить 'maxiter' или изменить параметры 'fatol'/'xatol' в опциях оптимизатора.")
        print("- Проверить правильность путей к данным и формат файлов.")

    print("\n--- Завершение работы ---")

# --- Запуск ---
if __name__ == "__main__":
    # Проверка существования директории с данными
    if not os.path.isdir(DATA_DIR):
         print(f"Ошибка: Директория с данными '{DATA_DIR}' не найдена.")
         print("Пожалуйста, создайте эту директорию, распакуйте в нее архив")
         print(f"и измените переменную DATA_DIR в скрипте, если нужно.")
    elif not os.path.isfile(KNOWN_FILTERS_FILE):
         print(f"Ошибка: Файл с известными фильтрами '{KNOWN_FILTERS_FILE}' не найден.")
         print(f"Убедитесь, что файл 'algos.csv' находится в папке '{DATA_DIR}'.")
    else:
        reconstruct_kainet(DATA_DIR, KNOWN_FILTERS_FILE, OUTPUT_CSV_FILE)

--- Начало восстановления KaiNet ---
Загружены 2 известных фильтра из ./best_pictures\algos.csv
Найдено 1000 пар вход/выход.
Загрузка данных для оптимизации (макс. 10 пар)...
  Загружена пара: 1.png / 1.txt
  Загружена пара: 10.png / 10.txt
  Загружена пара: 100.png / 100.txt
  Загружена пара: 1000.png / 1000.txt
  Загружена пара: 101.png / 101.txt
  Загружена пара: 102.png / 102.txt
  Загружена пара: 103.png / 103.txt
  Загружена пара: 104.png / 104.txt
  Загружена пара: 105.png / 105.txt
  Загружена пара: 106.png / 106.txt

Начинаем проверку 6 возможных порядков фильтров...

--- Проверка порядка #1: F1 -> F2 -> F_unknown ---
  Запуск оптимизации (Nelder-Mead) для поиска неизвестного фильтра...
  Оптимизация завершена. Минимальный MSE для этого порядка: 924.27540598
  *** Найден новый лучший результат! ***

--- Проверка порядка #2: F1 -> F_unknown -> F2 ---
  Запуск оптимизации (Nelder-Mead) для поиска неизвестного фильтра...
  Оптимизация завершена. Минимальный MSE для этого порядка:

# B

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import Model
import os
import csv
from sklearn.neighbors import NearestNeighbors
from PIL import Image
import warnings
import time

In [ ]:


# --- Configuration ---
# !!! ВАЖНО: Укажите путь к папке с изображениями !!!
IMAGE_DIR = './image_archive'  # Замените на ваш путь
OUTPUT_CSV_FILE = 'submission.csv'
NUM_NEIGHBORS = 6 + 1  # Ищем 6 похожих + 1 (само изображение)
# Целевой размер для ResNet50
IMG_WIDTH, IMG_HEIGHT = 224, 224
# Пакетная обработка для ускорения извлечения признаков
BATCH_SIZE = 32

# --- Helper Functions ---

def setup_gpu():
    """Настраивает использование GPU, если доступно."""
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Ограничиваем рост использования памяти GPU
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
        except RuntimeError as e:
            # Ошибки могут возникнуть, если память уже настроена
            print(e)
    else:
        print("No GPU detected, using CPU.")

def load_feature_extractor():
    """Загружает предобученную модель ResNet50 для извлечения признаков."""
    print("Загрузка предобученной модели ResNet50...")
    # Загружаем ResNet50 с весами ImageNet, без верхнего слоя (классификатора)
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg',
                          input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    # Мы используем выход слоя 'avg_pool' как вектор признаков
    # В данном случае base_model уже настроена с GlobalAveragePooling2D (pooling='avg')
    print("Модель загружена.")
    return base_model

def preprocess_image(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT)):
    """Загружает, изменяет размер и предобрабатывает изображение для ResNet50."""
    try:
        img = keras_image.load_img(img_path, target_size=target_size)
        img_array = keras_image.img_to_array(img)
        # Добавляем измерение для батча и применяем предобработку ResNet50
        img_array_expanded = np.expand_dims(img_array, axis=0)
        return preprocess_input(img_array_expanded)
    except Exception as e:
        warnings.warn(f"Ошибка при обработке изображения {img_path}: {e}. Пропуск.")
        return None

def extract_features_batch(model, image_paths):
    """Извлекает признаки для пакета изображений."""
    batch_features = []
    valid_indices = []
    processed_images = []

    for idx, img_path in enumerate(image_paths):
        processed = preprocess_image(img_path)
        if processed is not None:
            processed_images.append(processed[0]) # Убираем измерение батча
            valid_indices.append(idx)

    if not processed_images:
        return np.array([]), [] # Возвращаем пустой массив и пустой список индексов

    # Собираем валидные изображения в один батч
    batch_array = np.stack(processed_images, axis=0)

    # Извлекаем признаки
    features = model.predict(batch_array, verbose=0)
    return features, valid_indices


# --- Main Logic ---

def find_similar_images(image_dir, output_csv_file):
    """
    Главная функция для поиска похожих изображений.
    """
    print("--- Начало поиска похожих изображений ---")
    setup_gpu()

    # 1. Получение списка изображений
    try:
        all_files = [f for f in os.listdir(image_dir)
                     if os.path.isfile(os.path.join(image_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not all_files:
            print(f"Ошибка: В папке '{image_dir}' не найдено изображений.")
            return
        print(f"Найдено {len(all_files)} изображений.")
        # Сортируем для воспроизводимости порядка
        all_files.sort()
        image_paths = [os.path.join(image_dir, f) for f in all_files]
    except FileNotFoundError:
        print(f"Ошибка: Папка '{image_dir}' не найдена.")
        return
    except Exception as e:
        print(f"Ошибка при чтении директории '{image_dir}': {e}")
        return

    # 2. Загрузка модели
    try:
        feature_extractor = load_feature_extractor()
    except Exception as e:
        print(f"Критическая ошибка при загрузке модели: {e}")
        print("Убедитесь, что TensorFlow установлен и есть доступ к интернету для скачивания весов.")
        return

    # 3. Извлечение признаков для всех изображений (пакетная обработка)
    print("Извлечение признаков из изображений...")
    all_features = []
    valid_filenames = []
    start_time = time.time()

    for i in range(0, len(image_paths), BATCH_SIZE):
        batch_paths = image_paths[i:i+BATCH_SIZE]
        batch_filenames = all_files[i:i+BATCH_SIZE]

        features, valid_indices = extract_features_batch(feature_extractor, batch_paths)

        if features.size > 0:
            all_features.append(features)
            # Добавляем только имена файлов для успешно обработанных изображений
            valid_filenames.extend([batch_filenames[j] for j in valid_indices])

        processed_count = len(valid_filenames)
        if processed_count > 0:
             elapsed = time.time() - start_time
             eta = (elapsed / processed_count) * (len(all_files) - processed_count) if processed_count > 0 else 0
             print(f"  Обработано {processed_count}/{len(all_files)} изображений... (ETA: {eta:.1f} сек)", end='\r')

    print(f"\nИзвлечение признаков завершено за {time.time() - start_time:.2f} сек.")

    if not valid_filenames:
        print("Не удалось извлечь признаки ни из одного изображения.")
        return

    # Объединяем признаки из всех батчей
    feature_matrix = np.vstack(all_features)
    print(f"Размер итоговой матрицы признаков: {feature_matrix.shape}")

    # 4. Поиск ближайших соседей
    print(f"Поиск {NUM_NEIGHBORS-1} ближайших соседей для каждого изображения...")
    start_time = time.time()
    # Используем косинусное расстояние (ближе к 0 = более похожи)
    # algorithm='brute' может быть медленным, но точным для косинусного расстояния.
    # 'auto' может выбрать 'ball_tree' или 'kd_tree', которые могут быть не оптимальны для косинусной метрики.
    # Попробуем 'auto' сначала, если будет медленно, можно переключить на 'brute'.
    nn_model = NearestNeighbors(n_neighbors=NUM_NEIGHBORS, metric='cosine', algorithm='auto', n_jobs=-1)
    nn_model.fit(feature_matrix)
    # Ищем соседей для всех точек в матрице признаков
    distances, indices = nn_model.kneighbors(feature_matrix)
    print(f"Поиск соседей завершен за {time.time() - start_time:.2f} сек.")


    # 5. Формирование и сохранение результата
    print(f"Формирование и сохранение результатов в {output_csv_file}...")
    results = []
    for i in range(len(valid_filenames)):
        query_filename = valid_filenames[i]
        neighbor_indices = indices[i]
        neighbor_filenames = []
        for idx in neighbor_indices:
            # Исключаем само изображение из списка соседей
            if idx != i:
                neighbor_filenames.append(valid_filenames[idx])
            # Если набрали нужное количество, выходим (на случай дубликатов или ошибок)
            if len(neighbor_filenames) == NUM_NEIGHBORS - 1:
                break

        # Убедимся, что у нас ровно 6 соседей
        if len(neighbor_filenames) != NUM_NEIGHBORS - 1:
             warnings.warn(f"Для файла {query_filename} найдено {len(neighbor_filenames)} соседей вместо {NUM_NEIGHBORS - 1}. "
                           f"Возможно, есть дубликаты или проблемы с поиском.")
             # Дополняем пустыми строками или повторяем последнего соседа, если нужно (зависит от требований)
             # Пока просто оставим как есть, но это может потребовать уточнения.

        results.append({
            'filename': query_filename,
            'ranking': " ".join(neighbor_filenames)
        })

    # Запись в CSV
    try:
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['filename', 'ranking']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(results)
        print("Файл успешно сохранен.")
        print(f"\nПример первых строк файла {output_csv_file}:")
        with open(output_csv_file, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i > 5: break # Показываем заголовок + 5 строк данных
                print(line.strip())

    except Exception as e:
        print(f"Ошибка при сохранении файла {output_csv_file}: {e}")

    print("\n--- Завершение работы ---")


# --- Запуск ---
if __name__ == "__main__":
    # Проверка существования директории с данными
    if not os.path.isdir(IMAGE_DIR):
         print(f"Ошибка: Директория с изображениями '{IMAGE_DIR}' не найдена.")
         print("Пожалуйста, создайте эту директорию, распакуйте в нее архив")
         print(f"и измените переменную IMAGE_DIR в скрипте, если нужно.")
    else:
        find_similar_images(IMAGE_DIR, OUTPUT_CSV_FILE)
